In [1]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
# set seed to reproduce similar results
np.random.seed(0)

In [2]:
# 데이터 디렉토리 설정
data_dir = "C:/Users/wdd45/OneDrive/바탕 화면/딥러닝응용/human/UCI HAR Dataset/UCI HAR Dataset"

In [3]:
# 데이터 로딩 및 전처리
def load_HAR_data(data_dir):
    train_signals, train_labels, test_signals, test_labels = [], [], [], []

    for signal_type in ["train", "test"]:
        for signal in ["Inertial Signals/total_acc_x_", "Inertial Signals/body_acc_x_", "Inertial Signals/body_gyro_x_"]:
            filename = f"{data_dir}/{signal_type}/{signal}{signal_type}.txt"
            data = pd.read_csv(filename, delim_whitespace=True, header=None).values
            if "train" in signal_type:
                train_signals.append(data)
            else:
                test_signals.append(data)

        labels_filename = f"{data_dir}/{signal_type}/y_{signal_type}.txt"
        labels = pd.read_csv(labels_filename, header=None, names=["Activity"]).values
        if "train" in signal_type:
            train_labels.append(labels)
        else:
            test_labels.append(labels)

    train_signals = np.transpose(np.array(train_signals), (1, 2, 0))
    test_signals = np.transpose(np.array(test_signals), (1, 2, 0))
    train_labels = train_labels[0]
    test_labels = test_labels[0]

    return train_signals, train_labels, test_signals, test_labels

In [4]:
# 데이터 로딩
train_signals, train_labels, test_signals, test_labels = load_HAR_data(data_dir)

# 레이블 수정 (0부터 시작하도록)
train_labels = train_labels - 1
test_labels = test_labels - 1

In [5]:
# 데이터 정규화
train_signals = (train_signals - np.mean(train_signals)) / np.std(train_signals)
test_signals = (test_signals - np.mean(test_signals)) / np.std(test_signals)

In [6]:
# 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(train_signals, train_labels, test_size=0.2, random_state=42)

In [7]:
# 데이터를 적절하게 변환
X_train = X_train.transpose(0, 2, 1)
X_val = X_val.transpose(0, 2, 1)
test_signals = test_signals.transpose(0, 2, 1)

In [8]:
# 모델 아키텍처 정의 (입력 형태 수정)
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))  # pool_size를 (2,) 또는 (3,) 등으로 수정
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))  # 클래스 수에 따라 조정


In [9]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
# 모델 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 64)             24640     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 33,734
Trainable params: 33,734
Non-trainable params: 0
____________________________________________________

In [11]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
182/184 [============================>.] - ETA: 0s - loss: 0.7777 - accuracy: 0.6636WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000028AC5D12AF8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

In [12]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(test_signals, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

93/93 [==============================] - 0s 2ms/step - loss: 0.4527 - accuracy: 0.7954
Test accuracy: 79.54%
